In [ ]:
#Conduct analysis of docking results and analyze molecular structure trends in order to suggest possible next steps
#HarmonicFlow and inference are used interchangeably- analysis was run for HarmonicFlow output and with some checks/minor edits should be able to be run for other methods' output as well
import pandas as pd
import rdkit
from rdkit import Chem
from rdkit.Chem.rdchem import BondType as BT
from rdkit.Chem import AllChem, GetPeriodicTable, RemoveHs
from rdkit.Chem import Descriptors
from rdkit.Chem import rdDistGeom
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy
from scipy import stats

In [ ]:
#########
#Read molecule
#Copied from https://github.com/gcorso/DiffDock/blob/main/datasets/process_mols.py
#Inputs
#(a) molecule_file file with molecule
#also can specify whether to sanitize, find charge, and remove hydrogens
#Output
#(a) mol the molecule
#########
def read_molecule(molecule_file, sanitize=False, calc_charges=False, remove_hs=False):
    if molecule_file.endswith('.mol2'):
        mol = Chem.MolFromMol2File(molecule_file, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.sdf'):
        supplier = Chem.SDMolSupplier(molecule_file, sanitize=False, removeHs=False)
        mol = supplier[0]
    elif molecule_file.endswith('.pdbqt'):
        with open(molecule_file) as file:
            pdbqt_data = file.readlines()
        pdb_block = ''
        for line in pdbqt_data:
            pdb_block += '{}\n'.format(line[:66])
        mol = Chem.MolFromPDBBlock(pdb_block, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.pdb'):
        mol = Chem.MolFromPDBFile(molecule_file, sanitize=False, removeHs=False)
    else:
        raise ValueError('Expect the format of the molecule_file to be '
                         'one of .mol2, .sdf, .pdbqt and .pdb, got {}'.format(molecule_file))

    try:
        if sanitize or calc_charges:
            Chem.SanitizeMol(mol)

        if calc_charges:
            # Compute Gasteiger charges on the molecule.
            try:
                AllChem.ComputeGasteigerCharges(mol)
            except:
                warnings.warn('Unable to compute charges for the molecule.')

        if remove_hs:
            mol = Chem.RemoveHs(mol, sanitize=sanitize)
    except Exception as e:
        print(e)
        print("RDKit was unable to read the molecule.")
        return None

    return mol

In [ ]:
#########
#Find rmsd w/o pytorch
#Ref https://en.wikipedia.org/wiki/Root-mean-square_deviation_of_atomic_positions
#Ref https://github.com/HannesStark/FlowSite/blob/main/utils/train_utils.py
#Input:
#(a) c1 1st coords
#(b) c2 2nd coords
#Output: rmsd
#########
def rmsd_calc_no_torch(c1, c2):
    
    #Find difference in each atom's coordinates, find distance, square each distance
    #Sum squares, divide by atom count, take resulting square root
    coord_diff = c1 - c2
    coord_dist = np.linalg.norm(coord_diff, axis = 1)
    coord_dist_sq = coord_dist ** 2
    sum_sq = np.sum(coord_dist_sq)
    rmsd = (sum_sq / len(coord_diff)) ** 0.5
    return rmsd

In [ ]:
#########
#Convert to scientific notation to clean
#Copied this from https://stackoverflow.com/questions/29260893/convert-to-scientific-notation-in-python-a-×-10b
#Input:
#(a) number, number to convert
#Output: 
#(b) number in scientific notation format
#########
def sci_notation(number, sig_fig = 3):
    ret_string = "{0:.{1:d}e}".format(number, sig_fig)
    a, b = ret_string.split("e")
    b = int(b)
    #Ref https://stackoverflow.com/questions/21226868/superscript-in-python-plots for plot
    return a + "*$10^{" + str(b) + "}$"

In [ ]:
#########
#Calculate ligand properties
#Input:
#(a) lcalcpdb_no_h ligand from pdb, w/o Hs
#(b) lcalc_inf ligand from inference
#Output:
#(a) lprop_dict dictionary with keys of property names and values of properties
#########
def lprop_calc(lcalcpdb_no_h, lcalc_inf):
    
    #Keys are name of properties, and values are the properties
    lprop_dict = {}
    
    #Heavy atom and rotatable bond count
    #Ref https://www.rdkit.org/docs/source/rdkit.Chem.rdMolDescriptors.html
    lprop_dict["Heavy_Atom_Count"] = rdkit.Chem.rdMolDescriptors.CalcNumHeavyAtoms(lcalcpdb_no_h)
    lprop_dict["Rot_Bonds"] = rdkit.Chem.rdMolDescriptors.CalcNumRotatableBonds(lcalcpdb_no_h)
    
    #Radius of gyration
    #Ref https://www.rdkit.org/docs/source/rdkit.Chem.Descriptors3D.html
    rg_pdb = rdkit.Chem.Descriptors3D.RadiusOfGyration(lcalcpdb_no_h)
    lprop_dict["Rg_PDB"] = rg_pdb
    
    #Additional analyses if working with inference pose
    if lcalc_inf is not None:
        
        #Check heavy atom count is equal
        if rdkit.Chem.rdMolDescriptors.CalcNumHeavyAtoms(lcalcpdb_no_h) != rdkit.Chem.rdMolDescriptors.CalcNumHeavyAtoms(lcalc_inf):
            print("GetNumHeavyAtomsISSUE, different numbers in inference vs pdb")

        #Radius of gyration percent error
        rg_inf = rdkit.Chem.Descriptors3D.RadiusOfGyration(lcalc_inf)
        rg_pct_error = 100.0 * (rg_inf - rg_pdb) / rg_pdb
        lprop_dict["Rg_Inf"] = rg_inf
        lprop_dict["Rg_Percent_Error"] = rg_pct_error
    
    return lprop_dict

In [ ]:
#######
#Confirm atom ordering is correct
#Note this is not perfect - just a check of elements, not atom names or smarts
#Input:
#(a) hfc HF ligand checking
#(b) pdbc PDB ligand checking 
#######
def check_atoms(hfc, pdbc):

    #Element and atom id lists
    hf_at_el = []
    pdb_at_el = []
    hf_at_id = []
    pdb_at_id = []
    
    #Record atom info in each list
    #Get ID ref https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html, https://www.rdkit.org/docs/cppapi/classRDKit_1_1Atom.html
    #Could also explore https://www.rdkit.org/docs/source/rdkit.Chem.rdchem.html#rdkit.Chem.rdchem.Atom.GetProp
    for hfa in hfc.GetAtoms():
        hf_at_id.append(hfa.GetIdx())
        hf_at_el.append(hfa.GetAtomicNum())
    for pdba in pdbc.GetAtoms():
        pdb_at_id.append(pdba.GetIdx())
        pdb_at_el.append(pdba.GetAtomicNum())
    
    #Compare
    for ha, pa in zip(hf_at_id, pdb_at_id):
        if ha != pa:
            print("ISSUE WITH INDEXING-IDS")
    for he, pe in zip(hf_at_el, pdb_at_el):
        if ha != pa:
            print("ISSUE WITH INDEXING-ELEMENTS")

In [ ]:
#######
#2/1/24
#Distance calculation- revising because will do separately for HF
#Input:
#(a) lfordcalc ligand
#(b) posfordcalc positions
#(c) a1fordcalc atom 1 for calculation
#(d) a2fordcalc atom 2 for calculation
#Output
#(a) at1_atnum the atomic number of 1st atom
#(b) at2_atnum the atomic number of 2nd atom
#(c) calc_distance distance between atoms
#######
def dist_calc(lfordcalc, 
              posfordcalc, 
              a1fordcalc, 
              a2fordcalc):
            
    #Record elements
    at1_atnum = lfordcalc.GetAtoms()[a1fordcalc].GetAtomicNum()
    at2_atnum = lfordcalc.GetAtoms()[a2fordcalc].GetAtomicNum()

    #Positions and distance and record
    at_1_position = posfordcalc[a1fordcalc]
    at_2_position = posfordcalc[a2fordcalc]
    
    #Ref https://stackoverflow.com/questions/1401712/how-can-the-euclidean-distance-be-calculated-with-numpy
    calc_distance = np.linalg.norm(at_1_position - at_2_position)

    return at1_atnum, at2_atnum, calc_distance

In [ ]:
#######
#Find all distances of interest
#2/1/24 revise to call dist_calc so can work with pdb and inference/conformers separately
#Input:
#(a) confd conformer for distance calc
#(b) ligd ligand for distance calc
#(c) get_bonds_bool whether want to calculate distances from bonds
#(d) atom_pair_list atom pairs for distances- only specify if get_bonds_bool is False
#Output: 
#(a) d_dict distance info dictionary
#Ref https://www.rdkit.org/docs/GettingStartedInPython.html
#######
def obtain_dist_data(confd, 
                     ligd, 
                     get_bonds_bool, 
                     atom_pair_list):
    
    if (atom_pair_list is not None) and get_bonds_bool:
        print("obtain_dist_data input issue")
    if (atom_pair_list is None) and not get_bonds_bool:
        print("obtain_dist_data input issue")
        
    #Dictionary of keys of atom id pairs
    #Values of dictionaries with distances and elements
    d_dict = {}
    
    #Conformer coord pull
    cf_positions = confd.GetPositions()
    
    #Assemble list of atom ids in bonds if want to work w/bonds
    if get_bonds_bool:
        bond_list = []
        for b_rec in ligd.GetBonds():
            
            #Get atom IDs
            at1b = b_rec.GetBeginAtomIdx()
            at2b = b_rec.GetEndAtomIdx()
            bond_list.append(tuple(sorted([at1b, at2b])))
        
        pairs_calc = bond_list
        
    #Otherwise work with atom pairs
    else:
        pairs_calc = atom_pair_list
        
    #Iterate over pairs
    for ap in pairs_calc:
        
        #Sort and make possible dictionary entry
        d_key = tuple(sorted([ap[0], ap[1]]))
                      
        #For checking make 1st entry the lower atom id
        at1r = d_key[0]
        at2r = d_key[1]
        
        #Record elements and distance if heavy atoms involved
        an1, an2, dval = dist_calc(ligd, cf_positions, at1r, at2r)
        if an1 > 1 and an2 > 1:
            d_dict[d_key] = {}
            d_dict[d_key]["elements"] = [an1, an2]
            d_dict[d_key]["distance"] = dval
        else:
            print(f"HYDROGENISSUE {an1} and {an2} atomic numbers for {at1r} and {at2r}")
        
    return d_dict

In [ ]:
#########
#Calculate all bonded distance percent errors
#2/1/24 update to define set of bonds as all PDB bonds and only PDB bonds
#Ref https://github.com/rdkit/rdkit/issues/1982
#Ref https://www.rdkit.org/docs/GettingStartedInPython.html
#Input:
#(a) l_bpe_pdb ligand structure from pdb
#(b) l_bpe_inf ligand structure from inference
#Output:
#(a) pct_err_list list of each bond's percent error
#(b) pct_err_dict dictionary with bond percent error information
#(c) avg_bond_pe average bond percent error for the ligand
#########
def bond_percent_error_calc(l_bpe_pdb, l_bpe_inf):
    
    #Confirm comparing atoms correctly
    #Thank you (redact_for_anonymity) for also indicating in 1/30/24 conversation atom indexing should not change
    #So while this check is not perfect, its always succeeding plus (redact_for_anonymity) remarks imply going by atom ids should be alright
    check_atoms(l_bpe_inf, l_bpe_pdb)
    
    #Find all bond distances for inference and PDBbind
    #Ref https://github.com/HannesStark/EquiBind/blob/41bd00fd6801b95d2cf6c4d300cd76ae5e6dab5e/commons/process_mols.py helped process conformers
    distances_hf_list = []
    distance_pdb_list = []
    if len(l_bpe_inf.GetConformers()) > 1:
        print("HF conf PBM")
    if len(l_bpe_pdb.GetConformers()) > 1:
        print("PDB conf PBM")
    
    bd_pdb = obtain_dist_data(l_bpe_pdb.GetConformers()[0], l_bpe_pdb, True, None)
    pdb_bonds = list(bd_pdb.keys())
    bd_hf = obtain_dist_data(l_bpe_inf.GetConformers()[0], l_bpe_inf, False, pdb_bonds)

    #Find bonds in both dictionaries
    #Confirm same elements
    #Find distances and percent error
    bonds_in_only_hf = [b for b in list(bd_hf.keys()) if b not in list(bd_pdb.keys())]
    bonds_in_only_pdb = [b for b in list(bd_pdb.keys()) if b not in list(bd_hf.keys())]
    if (len(bonds_in_only_hf) > 0) or (len(bonds_in_only_pdb) > 0):
        print("BOND MISMATCH PROBLEM")
    pct_err_dict = {} #Dictionary of percent errors
    pct_err_list = [] #List of percent errors
    
    #Find percent error of each bond
    for bc in bd_pdb.keys():
        #Element check
        if (bd_hf[bc]["elements"][0] != bd_pdb[bc]["elements"][0]) or (bd_hf[bc]["elements"][1] != bd_pdb[bc]["elements"][1]):
            print("ELEMENT MISMATCH PROBLEM,inference and pdb compare")
            print(bc)
            print("HF")
            print(bd_hf[bc]["elements"])
            print("PDB")
            print(bd_pdb[bc]["elements"])

        #Percent error calc
        dist_pdb = bd_pdb[bc]["distance"]
        dist_hf = bd_hf[bc]["distance"]
        dist_pct_error = 100.0 * (dist_hf - dist_pdb) / dist_pdb
        pct_err_list.append(dist_pct_error)
        pct_err_dict[bc] = {"inf" : dist_hf, "pdb" : dist_pdb, "pe" : dist_pct_error}
        
    #Now average
    avg_bond_pe = np.average(pct_err_list)
    
    return pct_err_list, pct_err_dict, avg_bond_pe

In [ ]:
############
#Conformer generation
#2/1/24 13:40 update to redo rdkit generation if under 10 conformers, try w/random coords 1st
#Copied/slightly modified from https://github.com/HannesStark/EquiBind/blob/41bd00fd6801b95d2cf6c4d300cd76ae5e6dab5e/commons/process_mols.py#L447
#Thank you (redact_for_anonymity) for advice!
#Copied from ps down to conformers.append with some edits
#Also relevant/consulted:
#Ref https://iwatobipen.wordpress.com/2021/01/31/generate-conformers-script-with-rdkit-rdkit-chemoinformatics/
#Ref https://greglandrum.github.io/rdkit-blog/posts/2023-02-04-working-with-conformers.html
#Ref https://www.rdkit.org/docs/source/rdkit.Chem.rdDistGeom.html for random seed
#Ref https://github.com/rdkit/rdkit/issues/2575, https://github.com/rdkit/rdkit/discussions/6489, https://github.com/rdkit/rdkit/discussions/4623
#Ref https://github.com/rdkit/rdkit/discussions/5400 helped with figuring out a deterministic approach
#I think I only need 1 seed because I am generating all conformers at once
#Input:
#(a) l_c_gen molecule for conformer generation, believe needs Hs
#(b) cdir directory for saving conformer structures
#(c) cname complex name
#(d) count_conf how many conformers want
#Output:
#(a) conformers list of conformer coordinates
############
def generate_conformers(l_c_gen, 
                        cdir, 
                        cname, 
                        count_conf):

    ps = AllChem.ETKDGv2()
    
    #Update code so it will be deterministic unless error and to add more attempts
    #Ref https://github.com/rdkit/rdkit/issues/2536
    #Ref https://www.rdkit.org/docs/source/rdkit.Chem.rdDistGeom.html#rdkit.Chem.rdDistGeom.EmbedParameters
    ps.randomSeed = 5
    ps.maxAttempts = 10
    
    ids = rdDistGeom.EmbedMultipleConfs(l_c_gen, count_conf, ps)
    if (-1 in ids) or (len(list(ids)) < count_conf): #Edit here so this is now run if have under 10 conformers, 2/1/24 13:40
        print('rdkit coords could not be generated without using random coords. using random coords now.')
        ps.useRandomCoords = True
        ids = rdDistGeom.EmbedMultipleConfs(l_c_gen, count_conf, ps)
        AllChem.MMFFOptimizeMoleculeConfs(l_c_gen)
    else:
        AllChem.MMFFOptimizeMoleculeConfs(l_c_gen)
    conformers = []

    for i in range(count_conf):
        conformers.append(l_c_gen.GetConformer(i).GetPositions())
        
    #Save if want to
    #Copied writer code from https://stackoverflow.com/questions/69564484/how-to-save-rdkit-conformer-object-into-a-sdf-file
    if cdir is not None:
        cwriter = Chem.SDWriter(f"{cdir}/{cname}_confs.sdf")
        for cid in range(l_c_gen.GetNumConformers()):
            cwriter.write(l_c_gen, confId=cid)
        
    return conformers

In [ ]:
############
#Obtain long range (LR) distances from dictionary
#Input:
#(a) d_lr_calc dictionary for LR distance calc
#Output:
#(a) lr_p_list list of LR pairs
############
def lr_list_from_dict(d_lr_calc):
    
    #Initialize list
    lr_p_list = []
    
    #Each atom iterate
    for a_lr_1 in d_lr_calc.keys():
        
        #Each further away atom relative to it iterate
        for a_lr_2 in d_lr_calc[a_lr_1]:
            
            #Sorted pair
            pair_sorted = sorted((a_lr_1, a_lr_2))
            if pair_sorted not in lr_p_list:
                lr_p_list.append(pair_sorted)
                
    return lr_p_list

In [ ]:
############
#Work through bonds and find which pairs of atoms have 3+ bonds separating them
#Note need to confirm Hs are always at the end of the molecule, I confirmed for the 268 test set entries analyzed
#Input:
#(a) l_find_lr ligand for finding longer-range pairs
#(b) l_bonds list of ligand bonds
#Output:
#(a) lr_pairs list of pairs separated by 3+ bonds
############
def find_lr_pairs(l_find_lr, l_bonds):
    lr_pairs = []
    
    #A. Find all heavy atoms
    #Ref https://www.rdkit.org/docs/GettingStartedInPython.html
    heavy_atom_list = []

    #Record all heavy atoms
    for a in l_find_lr.GetAtoms():
        if a.GetAtomicNum() > 1:
            heavy_atom_list.append(a.GetIdx())
            
    #B. Dictionary of bonded atoms for each atom
    heavy_atom_bonds_dict = {}
    for ah in heavy_atom_list:
        
        #Atom is key, value is list of bonded atoms
        heavy_atom_bonds_dict[ah] = []
        
        #All bonds with atom- record other partner
        for bcheck in l_bonds:
            if ah in bcheck:
                if bcheck[0] == ah:
                    heavy_atom_bonds_dict[ah].append(bcheck[1])
                if bcheck[1] == ah:
                    heavy_atom_bonds_dict[ah].append(bcheck[0])
    
    #C Dictionary of 2 away for each atom
    heavy_atom_2_away_dict = {}
    
    for ah2 in heavy_atom_list:
        heavy_atom_2_away_dict[ah2] = []
        
        #Bonded neighbors
        for ah2_bp in heavy_atom_bonds_dict[ah2]:
            
            #Their bonded neighbors
            for ah2_bp_2 in heavy_atom_bonds_dict[ah2_bp]:
                
                #Do not add the atom itself and anything already in the bonded list or already in the 2 away list
                if (ah2 != ah2_bp_2) and (ah2_bp_2 not in heavy_atom_bonds_dict[ah2]) and (ah2_bp_2 not in heavy_atom_2_away_dict[ah2]):
                    heavy_atom_2_away_dict[ah2].append(ah2_bp_2)
    
    #D Dictionary of 3+ away for each atom
    heavy_atom_lr_dict = {}
    for ahlr in heavy_atom_list:
        
        #All 1 and 2 away plus atom itself
        close_list = heavy_atom_bonds_dict[ahlr] + heavy_atom_2_away_dict[ahlr] + [ahlr]
        
        #This list is everything except for what is 1 or 2 away
        heavy_atom_lr_dict[ahlr] = [hc for hc in heavy_atom_list if hc not in close_list]
    
    #E List from dictionary
    lr_pairs = lr_list_from_dict(heavy_atom_lr_dict)
    
    return lr_pairs

In [ ]:
############
#Determine which distances have a low enough standard deviation to be consensus distances
#Input
#(a) conf_sum_d dictionary with information for each distance's value in the conformer ensemble
#(b) consensus_sd_val threshold below which a distance is a consensus distance
#Output
#(a) consensus_d_list list of distances with low sds, representing consensus distances
#(b) len(consensus_d_list) how many distances are consensus ones
############
def find_consensus_dist(conf_sum_d, consensus_sd_val):
    consensus_d_list = []
    
    #Iterate over each distance, if sd is low enough add to list
    for lr_dist_check_c in conf_sum_d.keys():
        if conf_sum_d[lr_dist_check_c]["sd"] < consensus_sd_val:
            consensus_d_list.append(lr_dist_check_c)
            
    return consensus_d_list, len(consensus_d_list)

In [ ]:
############
#2/1/24 19:30 centralize analysis of conformers
#Generate conformers and determine consensus distances and % error relative to pdb
#Input:
#(a) l_for_conf ligand for conformer analysis - should have hydrogens ref https://greglandrum.github.io/rdkit-blog/posts/2023-02-04-working-with-conformers.html
#(b) dir_c directory for results
#(c) name_c name of complex
#(d) c_bonds bonded atom list of conformer
#(e) l_from_inference ligand from inference for % error calculation. It may not have Hs, so I run checks of atom numbering
#(f) list_sd_thresholds list of SD thresholds for consensus
#Output:
#(a) dict_of_sd_threshold_results consensus distance information for each sd cutoff. In particular this includes:
#(i) consensus_dist_count how many consensus distances exist
#(ii) consensus_dist_pe_list_conf list of percent error of consensus distances- conformer average relative to pdb value
#(iii) consensus_dist_pe_list_inf list of percent error of consensus distances- inference relative to pdb value
#(iv) consensus_dist_pe_dict dictionary with percent error info
#(b) all_pair_sd_list list of all pairs' standard deviations
############
def analyze_conformers(l_for_conf, 
                       dir_c, 
                       name_c, 
                       c_bonds, 
                       l_from_inference, 
                       list_sd_thresholds):
    
    if len(l_from_inference.GetConformers()) > 1:
        print("HF conf PBM")
    if len(l_for_conf.GetConformers()) > 1:
        print("PDB conf PBM")
    
    ############
    #A. Process structure
    ############
    #Find all heavy atom pairs 3 or more bonds away from each other
    heavy_atom_pairs_lr = find_lr_pairs(l_for_conf, c_bonds)
    
    if len(l_for_conf.GetConformers()) > 1:
        print("PDB conf PBM,for conf. gen.")
        
    #Find PDB pose distances for each pair
    pdb_lr_dist_dict = obtain_dist_data(l_for_conf.GetConformers()[0], 
                                        l_for_conf, 
                                        False, 
                                        heavy_atom_pairs_lr)
    
    ############
    #B. Generate conformers
    ############
    conf_num = 10
    conf = generate_conformers(l_for_conf, dir_c, name_c, conf_num)
    if len(conf) < 10:
        print("Issue:too few conformers")
     
    ############
    #C. Find long range distances for each conformer
    ############
    #Dictionary with conformers as key
    #Each conformer has its own dictionary
    #Values are the distance dictionaries from obtain_dist_data
    conf_lr_dist_dict = {}
    for c in range(len(conf)):
        conf_lr_dist_dict[c] = obtain_dist_data(l_for_conf.GetConformers()[c], 
                                                l_for_conf, 
                                                False,
                                                heavy_atom_pairs_lr)
      
    #Now create dictionary of summary stats for each long-range distance
    #Keys- atom pairs
    #Values- dictionaries with the distance for each conformer, plus the average and standard deviation
    conformer_summary_dict = {}
    
    #Add on list of all sds to show distribution
    all_pair_sd_list = []
    
    #Record info for each pair
    for ha_pair_lr in heavy_atom_pairs_lr:
        
        #Set up tuple
        ha_pair_sort = sorted(ha_pair_lr)
        ha_pair_sort_0 = ha_pair_sort[0]
        ha_pair_sort_1 = ha_pair_sort[1]
        ha_pair_lr_key = (ha_pair_sort_0, ha_pair_sort_1)
        
        #Dictionary setup
        conformer_summary_dict[ha_pair_lr_key] = {}
        
        #Find the distance of interest in each conformer
        conf_dist_list = [conf_lr_dist_dict[c_add][ha_pair_lr_key]["distance"] for c_add in conf_lr_dist_dict.keys()]
        
        #Also confirm elements match
        for c_el_check in conf_lr_dist_dict.keys(): #For each conformer
            
            #Find relevant elements
            c_el_check_lr_elements = conf_lr_dist_dict[c_el_check][ha_pair_lr_key]["elements"]
            
            #Compare to elements to PDB
            if (c_el_check_lr_elements[0] != pdb_lr_dist_dict[ha_pair_lr_key]["elements"][0]) or (c_el_check_lr_elements[1] != pdb_lr_dist_dict[ha_pair_lr_key]["elements"][1]):
                print("ELEMENT MISMATCH PROBLEM,conformers and pdb compare")
                print(ha_pair_lr)
                print("CONF")
                print(c_el_check_lr_elements)
                print("PDB")
                print(pdb_lr_dist_dict[ha_pair_lr_key]["elements"])
        
        #Record whole list, find summary statistics
        conformer_summary_dict[ha_pair_lr_key]["distance_list"] = conf_dist_list
        conformer_summary_dict[ha_pair_lr_key]["avg"] = np.average(conf_dist_list)
        sd_across_confs = np.std(conf_dist_list, ddof = 1)
        conformer_summary_dict[ha_pair_lr_key]["sd"] = sd_across_confs
        
        all_pair_sd_list.append(sd_across_confs)
    
    ############
    #D Find how many distances are consensus distances, find consensus distance % error for conformers
    ############
    #Find the consensus distances information- for different SD thresholds    
    #To hold results at different thresholds
    dict_of_sd_threshold_results = {}
    for consensus_sd_threshold in list_sd_thresholds:
        
        dict_of_sd_threshold_results[consensus_sd_threshold] = {}
        consensus_dist_list, consensus_dist_count = find_consensus_dist(conformer_summary_dict, consensus_sd_threshold)

        #Find percent error of each consensus distance
        consensus_dist_pe_dict = {}
        consensus_dist_pe_list_conf = []
        for cd in consensus_dist_list:
            conf_consensus_val = conformer_summary_dict[cd]["avg"]
            conf_consensus_sd = conformer_summary_dict[cd]["sd"]
            pdb_consensus_val = pdb_lr_dist_dict[cd]["distance"]
            pe_conf_pdb = 100.0 * (conf_consensus_val - pdb_consensus_val) / pdb_consensus_val
            consensus_dist_pe_dict[cd] = {"pdb" : pdb_consensus_val,
                                     "conformers" : conf_consensus_val,
                                     "pdb_conf_pe" : pe_conf_pdb,
                                     "conf_sd" : conf_consensus_sd}
            consensus_dist_pe_list_conf.append(pe_conf_pdb)


        ############
        #E For each consensus distance also find % error for inference
        ############    
        #Run check to make sure atom numbering is ok- that Hs are just at the end, heavy atom ordering is the same
        check_atoms(l_from_inference, l_for_conf)

        #Find the consensus distance atom pairs' distance values in the inference output
        inference_consensus_dist_dict = obtain_dist_data(l_from_inference.GetConformers()[0], 
                                                         l_from_inference, 
                                                         False,
                                                         consensus_dist_list)

        #List of consensus distance percent error values in inference pose
        consensus_dist_pe_list_inf = []

        #Repeat percent error calculation, but now compare inference- not conformer average- to PDB
        for icd in inference_consensus_dist_dict.keys():

            #Confirm elements are the same
            inference_elements = inference_consensus_dist_dict[icd]["elements"]
            pdb_elements = pdb_lr_dist_dict[icd]["elements"]
            if (inference_elements[0] != pdb_elements[0]) or (inference_elements[1] != pdb_elements[1]):
                print("ELEMENT MISMATCH PROBLEM,inference and pdb compare")
                print(icd)
                print("INFERENCE")
                print(inference_elements)
                print("PDB")
                print(pdb_elements)

            #Find inference distance and compute percent error relative to PDB
            inference_dist_value = inference_consensus_dist_dict[icd]["distance"]
            pdb_for_inf_pe = consensus_dist_pe_dict[icd]["pdb"]
            inf_pe_rel_to_pdb = 100.0 * (inference_dist_value - pdb_for_inf_pe) / pdb_for_inf_pe
            consensus_dist_pe_dict[icd]["inf"] = inference_dist_value
            consensus_dist_pe_dict[icd]["pdb_inf_pe"] = inf_pe_rel_to_pdb
            consensus_dist_pe_list_inf.append(inf_pe_rel_to_pdb)

        #Now record for this sd threshold
        dict_of_sd_threshold_results[consensus_sd_threshold]["consensus_dist_count"] = consensus_dist_count
        dict_of_sd_threshold_results[consensus_sd_threshold]["consensus_dist_pe_dict"] = consensus_dist_pe_dict
        dict_of_sd_threshold_results[consensus_sd_threshold]["consensus_dist_pe_list_conf"] = consensus_dist_pe_list_conf
        dict_of_sd_threshold_results[consensus_sd_threshold]["consensus_dist_pe_list_inf"] = consensus_dist_pe_list_inf
  
    return dict_of_sd_threshold_results, all_pair_sd_list

In [ ]:
#########
#Scatterplot
#(a) xdat x vals for points
#(b) ydat y vals for points
#(c) xlabel x axis label
#(d) ylabel y axis label
#(e) color_scatter color for plot
#(f) dsc dir for scatterplot
#(g) strsc string for scatterplot
#########
def create_scatter(xdat, 
                   ydat, 
                   xlabel, 
                   ylabel, 
                   color_scatter,
                   dsc,
                   strsc):
    
    fp, ap = plt.subplots()
    plt.scatter(xdat, ydat, alpha = 0.7, color = color_scatter)
    plt.xlabel(xlabel, fontsize = 24)
    plt.ylabel(ylabel, fontsize = 24)
    plt.xticks(fontsize = 24)
    plt.yticks(fontsize = 24)
    
    #Thank you (redact_for_anonymity) for Spearman R advice!
    #Ref https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.spearmanr.html
    #TODO remove p values or note not perfect for this context bcs have under 500 points- currently noting in text
    spearman_r = scipy.stats.spearmanr(xdat, ydat)
    spearman_r_sci_not = sci_notation(spearman_r[1])
    #Ref https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.text.html
    #Ref https://www.adamsmith.haus/python/answers/how-to-print-a-number-in-scientific-notation-in-python
    plt.text(1.02, 0.88, f"Spearman R : {spearman_r[0]:.3f}\np-value : {spearman_r_sci_not}", transform=ap.transAxes, fontsize = 20)
    
    #Ref https://stackoverflow.com/questions/12050393/how-to-force-the-y-axis-to-only-use-integers
    if xlabel in ["Heavy Atom Count", "Rotatable Bonds"]:
        ap.xaxis.get_major_locator().set_params(integer=True)
        
    if ylabel in ["Heavy Atom Count", "Rotatable Bonds"]:
        ap.yaxis.get_major_locator().set_params(integer=True)
        
    if xlabel == "Radius of Gyration % Error":
        plt.xticks(fontsize = 19)
    
    #Ref https://stackoverflow.com/questions/55942693/how-do-i-save-the-entire-graph-without-it-being-cut-off
    plt.savefig(f"{dsc}/Scatter_{strsc}.png", bbox_inches = "tight")
    
    plt.show()
    plt.close()
    plt.clf()
    plt.cla()

In [ ]:
#########
#Scatterplot- two y values
#Ref https://matplotlib.org/stable/gallery/subplots_axes_and_figures/secondary_axis.html
#Ref https://matplotlib.org/stable/gallery/subplots_axes_and_figures/two_scales.html
#(a) xdat x vals for points
#(b) ydat1 y vals for points 1st series
#(c) ydat2 y vals for points 2nd series
#(d) xlabel x axis label
#(e) ylabel1 y axis label 1st series
#(f) ylabel2 y axis label 2nd series
#(g) color_scatter1 color for plot 1st series
#(h) color_scatter2 color for plot 2nd series
#(i) dsc2 dir for double scatter
#(j) strsc2 string for double scatter
#########
def create_scatter_2_y(xdat, 
                       ydat1, 
                       ydat2, 
                       xlabel, 
                       ylabel1, 
                       ylabel2, 
                       color_scatter1, 
                       color_scatter2,
                       dsc2,
                       strsc2):
    
    fp, ap = plt.subplots()

    #One plot
    ap.scatter(xdat, ydat1, color = color_scatter1, alpha = 0.5)

    #Second
    ap2 = ap.twinx()
    ap2.scatter(xdat, ydat2, color = color_scatter2, alpha = 0.5)

    ap.set_xlabel(xlabel, fontsize = 24)
    if ylabel1 == "Radius of Gyration % Error":
        ylabel1_clean = "Rad. of Gyr. % Error"
    ap.set_ylabel(ylabel1_clean, fontsize = 24, color = color_scatter1)
    ap2.set_ylabel(ylabel2, fontsize = 24, color = color_scatter2)
    ap.tick_params(labelsize = 20)
    ap2.tick_params(labelsize = 20)
    y1_spearman = scipy.stats.spearmanr(xdat, ydat1)
    y1_spearman_p = sci_notation(y1_spearman[1])
    y2_spearman = scipy.stats.spearmanr(xdat, ydat2)
    y2_spearman_p = sci_notation(y2_spearman[1])
    
    #Tighter labels
    if xlabel == "Average Bond Distance % Error":
        xlabel_short = "Avg. BD %E"
    if ylabel1 == "Radius of Gyration % Error":
        ylabel1_short = "Rg %E"
    if ylabel2 == "RMSD (Å)":
        ylabel2_short = "RMSD"
    plt.text(1.23, 0.50, f"{xlabel_short} and {ylabel1_short}\nSpearman R : {y1_spearman[0]:.3f}\np-value : {y1_spearman_p}\n{xlabel_short} and {ylabel2_short}\nSpearman R : {y2_spearman[0]:.3f}\np-value : {y2_spearman_p}", transform=ap.transAxes, fontsize = 20)
    #Ref https://stackoverflow.com/questions/55942693/how-do-i-save-the-entire-graph-without-it-being-cut-off
    plt.savefig(f"{dsc2}/Double_Scatter_{strsc2}.png", bbox_inches = "tight")
    plt.show()
    
    plt.close()
    plt.clf()
    plt.cla()

In [ ]:
#########
#Histogram
#(a) hdat data for histogram
#(b) hlabel label for axis
#(c) hcolor color for plot
#(d) hbininf info for bins- dictionary
#(e) dhist dir for saving
#(f) strhist string for histogram
#########
def create_histogram(hdat, 
                     hlabel, 
                     hcolor, 
                     hbininf,
                     dhist,
                     strhist):
    hbins = np.arange(hbininf["min"], hbininf["max"], hbininf["bspace"])
    fhist, ahist = plt.subplots()
    plt.hist(hdat,
             bins = hbins,
             color = hcolor)
    
    plt.text(1.02, 0.50, "$μ$ " + f"{np.average(hdat):.1f}\n" + "$σ$ " + f"{np.std(hdat,ddof = 1):.1f}", transform = ahist.transAxes, fontsize = 20)

    #Ref https://stackoverflow.com/questions/12050393/how-to-force-the-y-axis-to-only-use-integers
    ahist.yaxis.get_major_locator().set_params(integer=True)
    
    #Ref https://www.delftstack.com/howto/matplotlib/how-to-place-legend-outside-of-the-plot-in-matplotlib/
    #Ref https://stackoverflow.com/questions/25068384/bbox-to-anchor-and-loc-in-matplotlib
    if hlabel == "Radius of Gyration % Error":
        plt.xticks(fontsize = 19)
    else:
        plt.xticks(fontsize = 24)
    plt.ylabel("Frequency", fontsize = 24)
    plt.yticks(fontsize = 24)
    plt.xlabel(hlabel, fontsize = 24)
    
    #Ref https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.axvline.html
    if hlabel in ["Radius of Gyration % Error", "Bond Distance % Error"]:
        plt.axvline(0, linestyle = "--", color = [0.3, 0.3, 0.3])
        
    #x lim to 0 if count dist.s or SDs
    #Ref https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xlim.html
    if hlabel in ["Count Consensus Distances", "All Long-Range Dist. Std. Dev. (Å)"]:
        plt.xlim(left = 0)
    
    #Ref https://stackoverflow.com/questions/55942693/how-do-i-save-the-entire-graph-without-it-being-cut-off
    plt.savefig(f"{dhist}/Histogram_{strhist}.png", bbox_inches = "tight")
    plt.show()
    
    plt.close()
    plt.clf()
    plt.cla()
    
    below_0 = [i for i in hdat if i < 0]
    print(f"{hlabel} has {len(hdat)} entries, {len(below_0)} below 0 for {100.0 * len(below_0) / len(hdat)}% below")

In [ ]:
#########
#Histogram with 2 entries
#(a) hdat_list list of data for histogram
#(b) hlabel label for axis
#(c) hcolor_list list of colors for plot
#(d) hbininf info for bins- dictionary
#(e) hdat_labels_list labels for histogram
#(f) norm_boolean whether to normalize- now do not have option- instead made separate function, but should merge
#(g) dhist2 dir for saving
#(h) strhist2 string for saving
#########
def create_histogram_2(hdat_list, 
                       hlabel, 
                       hcolor_list, 
                       hbininf, 
                       hdat_labels_list, 
                       norm_boolean,
                       dhist2,
                       strhist2):
    hbins = np.arange(hbininf["min"], hbininf["max"], hbininf["bspace"])
    fhist, ahist = plt.subplots()
    
    stats_string = ""
    
    for hd_i, hdat_pl in enumerate(hdat_list):
        plt.hist(hdat_pl,
                 bins = hbins,
                 color = hcolor_list[hd_i],
                 label = hdat_labels_list[hd_i],
                 alpha = 0.50)
        stats_for_entry = f"{hdat_labels_list[hd_i]}\n"+"$μ$ "+f"{np.average(hdat_pl):.1f}\n"+"$σ$ "+f"{np.std(hdat_pl,ddof = 1):.1f}\n"
        stats_string += stats_for_entry
    
    #Assume 2 entries, t-test
    #ref https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
    #TODO- is this the correct test?
    t_test_2_hist = scipy.stats.ttest_ind(hdat_list[0], hdat_list[1])
    t_test_2_hist_sci_not = sci_notation(t_test_2_hist[1])
    if t_test_2_hist[1] > 0.0:
        stats_string += f"t test p-value {t_test_2_hist_sci_not}"
    else:
        print("0 t test, not including")
    print(t_test_2_hist)
    
    plt.text(1.02, 0.27, stats_string, transform = ahist.transAxes, fontsize = 16)
    
    #Ref https://stackoverflow.com/questions/12050393/how-to-force-the-y-axis-to-only-use-integers
    ahist.yaxis.get_major_locator().set_params(integer=True)

    #Ref https://www.delftstack.com/howto/matplotlib/how-to-place-legend-outside-of-the-plot-in-matplotlib/
    #Ref https://stackoverflow.com/questions/25068384/bbox-to-anchor-and-loc-in-matplotlib
    plt.legend(bbox_to_anchor = (1.02, 1.00), loc = "upper left", fontsize = 15)
    plt.xlabel(hlabel, fontsize = 24)
    plt.ylabel("Frequency", fontsize = 24)
    plt.xticks(fontsize = 21)
    plt.yticks(fontsize = 24)
    
    #Ref https://stackoverflow.com/questions/55942693/how-do-i-save-the-entire-graph-without-it-being-cut-off
    plt.savefig(f"{dhist2}/Hist_2_{strhist2}.png", bbox_inches = "tight")
    
    plt.show()  
    plt.close()
    plt.clf()
    plt.cla()

In [ ]:
#########
#Create 1 list from list of lists
#Inputs:
#(a) list_to_consolidate list of lists
#Output:
#(a) consolidated_list list with all entries in each input list
#########
def list_consolidate(list_to_consolidate):
    
    consolidated_list = []
    
    for l_add in list_to_consolidate:
        consolidated_list += l_add
    
    return consolidated_list

In [ ]:
#########
#Create plots
#Input:
#(a) df_plot dataframe to plot
#(b) rlist list of entries to remove
#(c) pltd dir for saving
#(d) pltstri string with info
#########
def plot_data(df_plot, 
              rlist,
              pltd,
              pltstri):
    
    #########
    #Process, setup
    #########
    #Main color, thank you (redact_for_anonymity)!
    main_color = [0.8666666666666667, 0.5176470588235295, 0.3215686274509804]
    conf_color = [0.45, 0.65, 0.95]
    
    #Remove entries do not want
    #Filter removals
    #Ref https://saturncloud.io/blog/how-to-select-rows-from-a-dataframe-based-on-list-values-in-a-column-in-pandas/
    #Ref https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html
    df_removals = df_plot["Complex_Name"].isin(rlist)
    print("what is removed?")
    print(df_plot[df_removals])
    df_after_remove = df_plot[~df_removals]
    
    #Label, color setup
    df_to_label_dict = {"Heavy_Atom_Count" : "Heavy Atom Count",
                        "RMSD" : "RMSD (Å)",
                        "Rot_Bonds" : "Rotatable Bond Count",
                        "Rg_Percent_Error" : "Radius of Gyration % Error",
                        "Rg_Percent_Error_Abs_Val" : "|Radius of Gyration % Error|",
                        "Bond_Distance_Percent_Error_List" : "Bond Distance % Error",
                        "SD_0.10_Consensus_Distance_Count" : "Count Consensus Distances",
                        "All_LR_SD" : "All Long-Range Dist. Std. Dev. (Å)"}
    
    sc_color_plt_dict = {("Heavy_Atom_Count", "RMSD") : main_color,
                      ("Rot_Bonds", "RMSD") : main_color,
                      ("Rg_Percent_Error", "RMSD") : main_color,
                      ("Rg_Percent_Error", "Heavy_Atom_Count") : main_color,
                      ("Rg_Percent_Error_Abs_Val", "RMSD") : main_color,
                      ("Rg_Percent_Error_Abs_Val", "Heavy_Atom_Count") : main_color}
    
    #Construct Rg absolute value list
    rg_abs_val_list = [abs(rg_pe) for rg_pe in list(df_after_remove["Rg_Percent_Error"])]
    
    #########
    #Scatterplots
    #########
    #Scatterplots of drivers
    for dpair in [["Heavy_Atom_Count", "RMSD"], ["Rot_Bonds", "RMSD"], ["Rg_Percent_Error", "RMSD"], ["Rg_Percent_Error_Abs_Val", "RMSD"], ["Rg_Percent_Error", "Heavy_Atom_Count"], ["Rg_Percent_Error_Abs_Val", "Heavy_Atom_Count"]]:
        dp0 = dpair[0]
        dp1 = dpair[1]
        
        #Obtain x and y lists
        if dp0 != "Rg_Percent_Error_Abs_Val":
            x_rem = df_after_remove[dp0]
        else:
            x_rem = rg_abs_val_list
        y_rem = df_after_remove[dp1]
        
        #Plot
        create_scatter(x_rem, 
                       y_rem, 
                       df_to_label_dict[dp0], 
                       df_to_label_dict[dp1], 
                       sc_color_plt_dict[(dp0, dp1)], 
                       pltd,
                       f"{pltstri}_{dp0}_{dp1}")
        
    #Two y axis scatterplot
    create_scatter_2_y(df_after_remove["Bond_Distance_Percent_Error_Average"], 
                       df_after_remove["Rg_Percent_Error"], 
                       df_after_remove["RMSD"], 
                       "Average Bond Distance % Error", 
                       "Radius of Gyration % Error", 
                       "RMSD (Å)", 
                       [0.65, 0.65, 0.3], 
                       [0.17, 0.90, 0.25],
                       pltd,
                       f"{pltstri}_BDPE_RgPE_RMSD")
    #########
    #Histograms
    #########
    #Histogram setup
    #Extra step for some lists, to make 1 list out of all the lists
    consolidated_data_dict = {} #keys metrics, values consolidated lists
    keys_consolidate_list = ["Bond_Distance_Percent_Error_List", 
                              "All_LR_SD", 
                              "SD_0.10_Conformer_Consensus_Percent_Error",
                              "SD_0.10_Inference_Consensus_Percent_Error"]
    for kconsolidate in keys_consolidate_list:
        consolidated_data_dict[kconsolidate] = list_consolidate(list(df_after_remove[kconsolidate])) 
        print(kconsolidate)
        print(f"max {max(consolidated_data_dict[kconsolidate])} min {min(consolidated_data_dict[kconsolidate])}")
    
    #And now general setup
    hist_color_plt_dict = {"Rg_Percent_Error" : main_color,
                           "Bond_Distance_Percent_Error_List" :  main_color,
                           "SD_0.10_Consensus_Distance_Count" : conf_color,
                           "All_LR_SD" : conf_color,
                           "SD_0.10_Conformer_Consensus_Percent_Error" : conf_color,
                           "SD_0.10_Inference_Consensus_Percent_Error" : main_color}
    hist_bin_params = {
                        "Rg_Percent_Error" : {
                                                "min" : -60.0,
                                                "max" : 20.0,
                                                "bspace" : 2.0
                                             },
                        "Bond_Distance_Percent_Error_List" : {
                                                            "min" : -86.0,
                                                            "max" : 48.0, #was 160
                                                            "bspace" : 2.0
                                                         },
                         "SD_0.10_Consensus_Distance_Count" : {
                                                         "min" : 0,
                                                         "max" : 280,
                                                         "bspace" : 5
                                                      },
                        "All_LR_SD" : {
                                         "min" : 0.00,
                                         "max" : 6.00,
                                         "bspace" : 0.20
                                      }
                        
                      }
    
    #And create histogram
    for hplot in ["Rg_Percent_Error", "Bond_Distance_Percent_Error_List", "SD_0.10_Consensus_Distance_Count", "All_LR_SD"]:
        if hplot not in keys_consolidate_list:
            h_rem = df_after_remove[hplot]
            print(f"{hplot} maxhistval {max(list(h_rem))} minhistval {min(list(h_rem))}")
            create_histogram(h_rem, 
                             df_to_label_dict[hplot], 
                             hist_color_plt_dict[hplot], 
                             hist_bin_params[hplot],
                             pltd,
                             f"{pltstri}_{hplot}")
        else:
            if hplot in ["Bond_Distance_Percent_Error_List", "All_LR_SD"]:
                create_histogram(consolidated_data_dict[hplot],
                                 df_to_label_dict[hplot], 
                                 hist_color_plt_dict[hplot], 
                                 hist_bin_params[hplot],
                                 pltd,
                                 f"{pltstri}_{hplot}")
                
    #Add on histogram with 2 entries
    if len(consolidated_data_dict["SD_0.10_Conformer_Consensus_Percent_Error"]) != len(consolidated_data_dict["SD_0.10_Inference_Consensus_Percent_Error"]):
        print("PROBLEM, SD_0.10_Conformer_Consensus_Percent_Error and SD_0.10_Inference_Consensus_Percent_Error are diff. lengths")
    
    create_histogram_2([consolidated_data_dict["SD_0.10_Conformer_Consensus_Percent_Error"], 
                        consolidated_data_dict["SD_0.10_Inference_Consensus_Percent_Error"]],
                       "% error of non-bonded consensus dist.\n(relative to PDB)",
                       [conf_color, main_color], 
                       {"max" : 30.0, "min" : -60.0, "bspace" : 0.5}, 
                       ["Conformer Consensus", "HarmonicFlow"], 
                       False,
                       pltd,
                       f"{pltstri}_Consensus_Inf_PE")
    
    #Different axis limits
    create_histogram_2([consolidated_data_dict["SD_0.10_Conformer_Consensus_Percent_Error"], 
                        consolidated_data_dict["SD_0.10_Inference_Consensus_Percent_Error"]],
                       "% error of non-bonded consensus dist.\n(relative to PDB)",
                       [conf_color, main_color], 
                       {"max" : 60.0, "min" : -80.0, "bspace" : 0.5}, 
                       ["Conformer Consensus", "HarmonicFlow"], 
                       False,
                       pltd,
                       f"{pltstri}_Consensus_Inf_PE_All_Data")

In [ ]:
#########
#From earlier workflow
#Create normalized data
#Ref https://stackoverflow.com/questions/53063231/matplotlib-pyplot-hist-wrong-normed-property?noredirect=1&lq=1
#Ref https://numpy.org/doc/stable/reference/generated/numpy.histogram.html
#Ref https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html
#Ref https://stackoverflow.com/questions/22241240/plot-a-histogram-such-that-the-total-height-equals-1
#Ref https://stackoverflow.com/questions/18082536/numpy-histogram-normalized-with-specified-edges-python
#Ref https://stackoverflow.com/questions/3866520/plot-a-histogram-such-that-bar-heights-sum-to-1-probability/16399202#16399202
#Create histogram data to normalize
#Input:
#(a) dplot data to plot
#(b) bplot bins to plot
#Output:
#(a) hist_for_pl_norm histogram for plot
#########
def gen_hist_data(dplot, bplot):
    hist_for_pl, bins_for_pl = np.histogram(dplot, density = True, bins= bplot)
    hist_for_pl_sum = hist_for_pl.sum()
    hist_for_pl_norm = hist_for_pl / hist_for_pl_sum
    return hist_for_pl_norm

In [ ]:
#########
#Plot the 3 consensus count distributions on 1 axis
#Now SD values fixed, should edit to be more flexible
#Input:
#(a) check_df_sd_consensus_pl dataframe for plots
#(b) dhist3 directory to save
#########
def plt_consensus_hist(check_df_sd_consensus_pl, dhist3):
    
    #Setup
    bins_consensus = np.arange(0, 350, 5)
    alpha_pl = 0.50
    fhist, ahist = plt.subplots()
    
    #Plot
    plt.hist(check_df_sd_consensus_pl['SD_0.05_Consensus_Distance_Count'], 
             bins = bins_consensus, 
             alpha = alpha_pl, 
             label = "0.05 Å cutoff", 
             color = [0.45, 0.95, 0.65])
    plt.hist(check_df_sd_consensus_pl['SD_0.10_Consensus_Distance_Count'], 
             bins = bins_consensus, 
             alpha = alpha_pl, 
             label = "0.10 Å cutoff", 
             color = [0.45, 0.65, 0.95])
    plt.hist(check_df_sd_consensus_pl['SD_0.15_Consensus_Distance_Count'], 
             bins = bins_consensus, 
             alpha = alpha_pl, 
             label = "0.15 Å cutoff", 
             color = [0.95, 0.45, 0.65])

    #Summary stats
    sd_consensus_str = ""
    for cval in [0.05, 0.10, 0.15]:
        key_str_c = f'SD_{cval:.2f}_Consensus_Distance_Count'
        sd_consensus_str += f"{cval:.2f} Å "+"$μ$ "+f"{np.average(check_df_sd_consensus_pl[key_str_c]):.1f}"+" $σ$ "+f"{np.std(check_df_sd_consensus_pl[key_str_c],ddof = 1):.1f}\n"

    plt.text(1.02, 0.27, sd_consensus_str, transform = ahist.transAxes, fontsize = 16)

    #Ref https://www.delftstack.com/howto/matplotlib/how-to-place-legend-outside-of-the-plot-in-matplotlib/
    #Ref https://stackoverflow.com/questions/25068384/bbox-to-anchor-and-loc-in-matplotlib
    plt.legend(bbox_to_anchor = (1.02, 1.00), loc = "upper left", fontsize = 15)
    plt.xlabel("Count Consensus Distance", fontsize = 20)
    plt.ylabel("Frequency", fontsize = 20)
    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    
    #x lim to 0 
    #Ref https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xlim.html
    plt.xlim(left = 0)

    #Ref https://stackoverflow.com/questions/55942693/how-do-i-save-the-entire-graph-without-it-being-cut-off
    plt.savefig(f"{dhist3}/Hist_3_Consensus.png", bbox_inches = "tight")

    plt.show()  
    plt.close()
    plt.clf()
    plt.cla()

In [ ]:
#########
#From earlier workflow
#Test and training data histogram
#Input:
#(a) train_data training data vals
#(b) test_data test data vals
#(c) maxval max for bins
#(d) minval min for bins
#(e) binval increment for bins
#(f) pltxlabel x label for plot
#(g) dhist_tt directory for saving
#(h) strhist_tt string for plot
#########
def histogram_train_test(train_data, 
                         test_data,
                         maxval, 
                         minval, 
                         binval,
                         pltxlabel,
                         dhist_tt,
                         strhist_tt):
    
    avalh = 0.3
    hist_bins = np.arange(minval, maxval, binval)
    train_data_h = gen_hist_data(train_data, hist_bins)
    test_data_h = gen_hist_data(test_data, hist_bins)
    
    fhist, ahist = plt.subplots()
    plt.hist(hist_bins[:-1],
         bins = hist_bins,
         weights = test_data_h,
         color = [0.8666666666666667, 0.5176470588235295, 0.3215686274509804], #thank you (redact_for_anonymity) for the color!
         alpha = avalh,
         label = "Test")    
    plt.hist(hist_bins[:-1],
         bins = hist_bins,
         weights = train_data_h,
         color = [0.92, 0.06, 0.87],
         alpha = avalh,
         label = "Train")

    
    #Run t test
    #Ref https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html
    #TODO- is this the correct test?
    t_test_tr_test = scipy.stats.ttest_ind(train_data, test_data)
    #Ref https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.text.html
    #Ref https://www.adamsmith.haus/python/answers/how-to-print-a-number-in-scientific-notation-in-python
    #Code for sci. notation
    t_test_tr_test_sci_not = sci_notation(t_test_tr_test[1])
    #Ref https://stackoverflow.com/questions/21226868/superscript-in-python-plots for plot
    #Ref https://stackoverflow.com/questions/21226868/superscript-in-python-plots
    plt.text(1.02, 0.10, f"t test p-value:\n{t_test_tr_test_sci_not}\n"+"$μ_{train}$"+f" {np.average(train_data):.1f}\n"+"$σ_{train}$" + f" {np.std(train_data,ddof = 1):.1f}\n"+"$μ_{test}$"+f" {np.average(test_data):.1f}\n"+"$σ_{test}$"+f" {np.std(test_data,ddof = 1):.1f}", transform = ahist.transAxes, fontsize = 20)

    #Ref https://www.delftstack.com/howto/matplotlib/how-to-place-legend-outside-of-the-plot-in-matplotlib/
    plt.legend(bbox_to_anchor = (1.02, 1.00), loc = "upper left", fontsize = 20)
    plt.xlabel(pltxlabel, fontsize = 24)
    plt.ylabel("Norm. Frequency", fontsize = 24)
    plt.xticks(fontsize = 24)
    plt.yticks(fontsize = 24)
    
    if pltxlabel == "Heavy Atom Count":
        plt.xticks(fontsize = 20)
    
    #x lim to 0 
    #Ref https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.xlim.html
    plt.xlim(left = 0)
    
    #Ref https://stackoverflow.com/questions/55942693/how-do-i-save-the-entire-graph-without-it-being-cut-off
    plt.savefig(f"{dhist_tt}/Hist_Test_Train_{strhist_tt}.png", bbox_inches = "tight")
    
    plt.show()
    plt.close()
    plt.clf()
    plt.cla()

In [ ]:
#########
#Obtain list from a file
#Input:
#(a) f_name file name
#Output:
#(a) list_from_file list from the file
#########
def obtain_list_from_file(f_name):
    list_from_file = []
    file_for_list = open(f_name, "r")
    for entry_add in file_for_list:
        list_from_file.append(entry_add.rstrip())
    file_for_list.close()
    return list_from_file

In [ ]:
#########
#Launch the analysis
#Input:
#(a) pdb_dir the directory with PDBbind data
#(b) inf_dir directory with inference data
#(c) dir_output the directory for output
#(d) complex_list the list of complexes to analyze
#(e) boolean_sanitize whether to sanitize molecules
#Output
#(a) pd_df output dataframe
#########
def run_analysis(pdb_dir,
                 inf_dir,
                 dir_output,
                 complex_list,
                 boolean_sanitize):

    #Make directory
    os.mkdir(dir_output)
    c_dir_output = f"{dir_output}/Conformers"
    os.mkdir(c_dir_output)
    
    #Job info
    j_info = open(f"{dir_output}/Job_info.txt", "w")
    j_info.write(f"PDB : {pdb_dir}\nInf: {inf_dir}\nSanitize: {boolean_sanitize}\nComplexes\n")
    for co in complex_list:
        j_info.write(f"{co}\n")
    j_info.close()
    
    #List if there are 2+ molecules in the file
    mult_molec_in_file = []
    
    #Metrics for both only pdb (train) and inference + pdb (test) runs
    list_pdb_names = [] #Complex names
    list_hac = [] #Count of ligand heavy atoms
    list_rot_bonds = [] #Count of rotatable bonds
    list_rg_pdb = [] #PDB ligand radius of gyration
    
    #If this is a test set also use other mostly pose-based metrics/more in-depth analysis
    #Some of the consensus analyses could also be run for entire training set but for consistency not yet doing
    if inf_dir is not None:

        #Lists will use for dataframe
        list_rmsds = [] #RMSD- of inference 1 sample last entry in xt.pdb to PDBbind
        list_rg_inf = [] #Inference pose radius of gyration
        list_rg_pct_error = [] #Radius of gyration percent error
        list_b_dist_pe_all = [] #Bond distance percent error list- each entry is a list of all percent errors for that complex
        list_b_dist_pe_dict = [] #Bond distance percent error dictionary list- each entry is a dictionary with information on each bond used for its percent error calculation
        list_b_dist_pe_avg = [] #Average bond distance percent error list- each entry is the average of all bonds' percent errors for that complex
        list_lr_sd_all = [] #Every long range SD
        
        #Consensus results dictionary for each SD threshold
        #Keys are the thresholds, values are dictionaries with the metrics
        #Note right now there is repetition - a higher SD will also have the results for lower SD thresholds plus those in between the next lowest threshold and this threshold
        #Definitely a point for further cleaning
        sd_threshold_dict = {}
        sd_threshold_dict_keys = [0.05, 0.10, 0.15]
        for sd_t_use in sd_threshold_dict_keys:
            sd_threshold_dict[sd_t_use] = {
                                            "list_consensus_dist_count" : [], #How many consensus distances in each complex
                                            "list_consensus_pe_conf_list" : [], #Percent errors of conformer consensus relative to PDB
                                            "list_consensus_pe_inf_list" : [], #Percent errors of inference relative to PDB
                                            "list_consensus_dict" : [] #Dictionaries with consensus distance info
                                          }
            

        

    #Check each complex
    for ip, pdb_name in enumerate(complex_list):

        #Progress track
        print(f"On {pdb_name} index {ip}")
        #Copied below from https://github.com/gcorso/DiffDock/blob/main/datasets/pdbbind.py
        for file in os.listdir(os.path.join(pdb_dir, pdb_name)):
            if file.endswith(".sdf") and 'rdkit' not in file:

                lig = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=False, sanitize=boolean_sanitize)
                #Pull w/o H for rmsd comparison w/HF output
                lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file), remove_hs=True, sanitize=boolean_sanitize)
                if lig_no_h is None and os.path.exists(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2")):  # read mol2 file if sdf file cannot be sanitized
                    print('Using the .sdf file failed. We found a .mol2 file instead and are trying to use that.')
                    lig = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=False, sanitize=boolean_sanitize)
                    lig_no_h = read_molecule(os.path.join(pdb_dir, pdb_name, file[:-4] + ".mol2"), remove_hs=True, sanitize=boolean_sanitize)


                #Parse to see if there are 2+ molecules. If so- then remove from analysis
                #ref https://www.rdkit.org/docs/GettingStartedInPython.html#molecular-fragments
                #ref https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html
                l_smiles = Chem.MolToSmiles(lig)
                if "." in l_smiles:
                    mult_molec_in_file.append(pdb_name)
                    print(f"PDB {pdb_name} Ligand {l_smiles} multiple molecules, do not use")
                    
                #If 1 molecule continue w/analysis
                else:
                    #Record in list
                    list_pdb_names.append(pdb_name)
                    
                    #Also read in inference pose if working with a test set
                    #TODO does sanitize=False affect properties of interest? based on checks thus far no
                    if inf_dir is not None:
                        lighf = read_molecule(f"{inf_dir}/{pdb_name}_x20.pdb", remove_hs=True, sanitize=False)

                        #Pull positions
                        #Ref https://github.com/HannesStark/EquiBind/blob/41bd00fd6801b95d2cf6c4d300cd76ae5e6dab5e/commons/process_mols.py#L447 helped process molecules
                        lig_coord = lig_no_h.GetConformer(0).GetPositions()
                        hf_coord = lighf.GetConformer(0).GetPositions()

                        #Confirm atom count same, sometimes Hs are included
                        count_l_atoms = len(lig_no_h.GetAtoms())
                        count_lhf_atoms = len(lighf.GetAtoms())

                        #If Hs are included only use non-H atom coordinates
                        if count_l_atoms != count_lhf_atoms:

                            #New coordinate list, iterate over atoms, only record if not hydrogen
                            lig_coord_cleaned = []
                            for ac, cc in zip(lig_no_h.GetAtoms(), lig_coord):
                                if ac.GetSymbol() != "H":
                                    lig_coord_cleaned.append(list(cc))

                        else:
                            lig_coord_cleaned = lig_coord

                        #RMSD calculation
                        rmsd_calc = rmsd_calc_no_torch(lig_coord_cleaned, hf_coord)
                        list_rmsds.append(rmsd_calc)
                    else:
                        lighf = None

                    #Property calculation
                    prop_calc = lprop_calc(lig_no_h, lighf)

                    #Update lists
                    list_hac.append(prop_calc["Heavy_Atom_Count"])
                    list_rot_bonds.append(prop_calc["Rot_Bonds"])
                    list_rg_pdb.append(prop_calc["Rg_PDB"])

                    #If working with test set update additional lists, run bond distance % error and consensus distance analysis
                    if lighf is not None:
                        list_rg_pct_error.append(prop_calc["Rg_Percent_Error"])
                        list_rg_inf.append(prop_calc["Rg_Inf"])

                        #Bond percent error
                        bdist_l, bdist_d, bdist_avg = bond_percent_error_calc(lig_no_h, lighf)

                        #Update lists
                        list_b_dist_pe_all.append(bdist_l)
                        list_b_dist_pe_dict.append(bdist_d)
                        list_b_dist_pe_avg.append(bdist_avg)

                        #Conformer analysis
                        consensus_sd_output_dict, list_all_sd = analyze_conformers(lig, 
                                                                                   c_dir_output, 
                                                                                   pdb_name, 
                                                                                   list(bdist_d.keys()), 
                                                                                   lighf,
                                                                                   list(sd_threshold_dict.keys()))
                        
                        #Have multiple thresholds, update each dictionary
                        for sd_update in sd_threshold_dict.keys():
                            sd_threshold_dict[sd_update]["list_consensus_dist_count"].append(consensus_sd_output_dict[sd_update]["consensus_dist_count"])
                            sd_threshold_dict[sd_update]["list_consensus_pe_conf_list"].append(consensus_sd_output_dict[sd_update]["consensus_dist_pe_list_conf"])
                            sd_threshold_dict[sd_update]["list_consensus_pe_inf_list"].append(consensus_sd_output_dict[sd_update]["consensus_dist_pe_list_inf"])
                            sd_threshold_dict[sd_update]["list_consensus_dict"].append(consensus_sd_output_dict[sd_update]["consensus_dist_pe_dict"])
                        
                        #Update list of all LR SDs
                        list_lr_sd_all.append(list_all_sd)

    #Now construct dataframe
    chem_prop_dict = {
                        "Complex_Name" : list_pdb_names,
                        "Heavy_Atom_Count" : list_hac,
                        "Rot_Bonds" : list_rot_bonds,
                        "Rg_PDB" : list_rg_pdb,
                     }
    
    #Additional properties if working with inference output
    if lighf is not None:
        chem_prop_dict["RMSD"] = list_rmsds
        chem_prop_dict["Rg_Inf"] = list_rg_inf
        chem_prop_dict["Rg_Percent_Error"] = list_rg_pct_error
        chem_prop_dict["Bond_Distance_Percent_Error_List"] = list_b_dist_pe_all
        chem_prop_dict["Bond_Distance_Percent_Error_Dict"] = list_b_dist_pe_dict
        chem_prop_dict["Bond_Distance_Percent_Error_Average"] = list_b_dist_pe_avg
        chem_prop_dict["All_LR_SD"] = list_lr_sd_all
        
        #Each SD threshold has its own entries
        for sd_for_df in sd_threshold_dict.keys():
            chem_prop_dict[f"SD_{sd_for_df:.2f}_Consensus_Distance_Count"] = sd_threshold_dict[sd_for_df]["list_consensus_dist_count"]
            chem_prop_dict[f"SD_{sd_for_df:.2f}_Conformer_Consensus_Percent_Error"] = sd_threshold_dict[sd_for_df]["list_consensus_pe_conf_list"]
            chem_prop_dict[f"SD_{sd_for_df:.2f}_Inference_Consensus_Percent_Error"] = sd_threshold_dict[sd_for_df]["list_consensus_pe_inf_list"]
            chem_prop_dict[f"SD_{sd_for_df:.2f}_Consensus_Info_Dict"] = sd_threshold_dict[sd_for_df]["list_consensus_dict"]
        

    #Ref https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html
    pd_df = pd.DataFrame(data = chem_prop_dict)
    #Save out csv
    #Ref https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html
    pd_df.to_csv(f"{dir_output}/Chem_Prop_Data.csv")
    
    #Save out multiple molecule info
    np.save(f"{dir_output}/Multiple_Molec_List.npy", mult_molec_in_file)

    #Plot
    if lighf is not None:
        plot_data(pd_df, [], dir_output, "From_Script_Run")

    return pd_df

In [ ]:
#Test set analysis
#This has processed PDBbind data
#From https://github.com/HannesStark/FlowSite/tree/main Zenodo link
pdb_dir_check = "PDBbind_dir"

#This has parsed _xt.pdb output, the 20th and last set of coordinates for each file, in one directory
#To obtain this, the raw _xt.pdb output should have its last set of coordinates extracted to a file titled {namepdb}_x20.pdb
#All such files should be in one directory
hf_out_dir_check = "output_pose_dir"

#List of PDB ids of test set compounds
test_list = obtain_list_from_file("test_set_list.txt")

#Whether to sanitize
s_boolean = True

#Output directory
dir_output_run = "analysis_output_dir"
check_df_sd_consensus_new_c_2 = run_analysis(pdb_dir_check, 
                                     hf_out_dir_check, 
                                     dir_output_run,
                                     test_list,
                                     s_boolean)

In [ ]:
#Run for training set too
#This has processed PDBbind data
pdb_dir_check = "PDBbind_dir"

#List of PDB ids of training set compounds
train_set_list = obtain_list_from_file("training_set_list.txt")

#Output directory
dir_output_run_train = f"{dir_output_run}_Training_Set"
check_df_training = run_analysis(pdb_dir_check, 
                        None, 
                        dir_output_run_train,
                        train_set_list,
                        s_boolean)

In [ ]:
#Check max/min
print(f"Heavy_Atom_Count max {max(list(check_df_training['Heavy_Atom_Count']))} {max(list(check_df_sd_consensus_new_c_2['Heavy_Atom_Count']))}")
print(f"Rot_Bonds max {max(list(check_df_training['Rot_Bonds']))} {max(list(check_df_sd_consensus_new_c_2['Rot_Bonds']))}")
print(f"Rg max {max(list(check_df_training['Rg_PDB']))} {max(list(check_df_sd_consensus_new_c_2['Rg_PDB']))}")
print(f"SD consensus max {max(list(check_df_sd_consensus_new_c_2['SD_0.15_Consensus_Distance_Count']))}")

In [ ]:
#Output directory for additional plots
dir_output_additional_plots = f"{dir_output_run}_Additional_Plots"
os.mkdir(dir_output_additional_plots)

#Test/train plots
histogram_train_test(check_df_training["Heavy_Atom_Count"], 
                         check_df_sd_consensus_new_c_2["Heavy_Atom_Count"],
                         170.0, 
                         0.0, 
                         2.0,
                         "Heavy Atom Count",
                         dir_output_additional_plots,
                         "Heavy_Atom_Count")

histogram_train_test(check_df_training["Rot_Bonds"], 
                         check_df_sd_consensus_new_c_2["Rot_Bonds"],
                         60.0, 
                         0.0, 
                         1.0,
                         "Rotatable Bond Count",
                         dir_output_additional_plots,
                         "Rot_Bonds")

histogram_train_test(check_df_training["Rg_PDB"], 
                         check_df_sd_consensus_new_c_2["Rg_PDB"],
                         14.0, 
                         0.0, 
                         0.2,
                         "Radius of Gyration",
                         dir_output_additional_plots,
                         "Rg_PDB")

In [ ]:
#Consensus distances for different thresholds
plt_consensus_hist(check_df_sd_consensus_new_c_2, 
                   dir_output_additional_plots)

In [ ]:
#Obtain count of complexes with 5+ consensus distances
print(len([i for i in check_df_sd_consensus_new_c_2["SD_0.10_Consensus_Distance_Count"] if i > 4]))

In [ ]:
#Max min check
consol_05 = list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.05_Conformer_Consensus_Percent_Error"]))
consol_10 = list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.10_Conformer_Consensus_Percent_Error"]))
consol_15 = list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.15_Conformer_Consensus_Percent_Error"]))
print("conformer consensus")
print(f"consol_05 max {max(consol_05)} min {min(consol_05)} len {len(consol_05)}")
print(f"consol_10 max {max(consol_10)} min {min(consol_10)} len {len(consol_10)}")
print(f"consol_15 max {max(consol_10)} min {min(consol_15)} len {len(consol_15)}")

#Inference max min check
consol_05_i = list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.05_Inference_Consensus_Percent_Error"]))
consol_10_i = list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.10_Inference_Consensus_Percent_Error"]))
consol_15_i = list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.15_Inference_Consensus_Percent_Error"]))
print("inference")
print(f"consol_05 max {max(consol_05_i)} min {min(consol_05_i)} len {len(consol_05_i)}")
print(f"consol_10 max {max(consol_10_i)} min {min(consol_10_i)} len {len(consol_10_i)}")
print(f"consol_15 max {max(consol_10_i)} min {min(consol_15_i)} len {len(consol_15_i)}")

In [ ]:
#Different threshold percent errors
maxvalhpe = 30.0
minvalhpe = -60.0
ccolor = [0.45, 0.65, 0.95] 
icolor = [0.8666666666666667, 0.5176470588235295, 0.3215686274509804]
create_histogram_2([list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.05_Conformer_Consensus_Percent_Error"])), 
                list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.05_Inference_Consensus_Percent_Error"]))],
               "% error of non-bonded consensus dist.\n(relative to PDB, 0.05 Å cutoff)",
               [ccolor, icolor], 
               {"max" : maxvalhpe, "min" : minvalhpe, "bspace" : 0.5}, 
               ["Conformer Consensus", "HarmonicFlow"], 
               False,
               dir_output_additional_plots,
               "0.05A_Consensus_Inf_PE")

create_histogram_2([list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.15_Conformer_Consensus_Percent_Error"])), 
                list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.15_Inference_Consensus_Percent_Error"]))],
               "% error of non-bonded consensus dist.\n(relative to PDB, 0.15 Å cutoff)",
               [ccolor, icolor], 
               {"max" : maxvalhpe, "min" : minvalhpe, "bspace" : 0.5}, 
               ["Conformer Consensus", "HarmonicFlow"], 
               False,
               dir_output_additional_plots,
               "0.15A_Consensus_Inf_PE")

create_histogram_2([list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.10_Conformer_Consensus_Percent_Error"])), 
                list_consolidate(list(check_df_sd_consensus_new_c_2["SD_0.10_Inference_Consensus_Percent_Error"]))],
               "% error of non-bonded consensus dist.\n(relative to PDB, 0.10 Å cutoff)",
               [ccolor, icolor], 
               {"max" : maxvalhpe, "min" : minvalhpe, "bspace" : 0.5}, 
               ["Conformer Consensus", "HarmonicFlow"], 
               False,
               dir_output_additional_plots,
               "0.10A_Consensus_Inf_PE")